In [1]:
# This program produces the data for Table 1 and is designed to be run on a virtual machine with greater computing power. 
# This specific code took about 24 hours to run on a machine with 12 CPUs.

In [ ]:
using Distributions
using Plots
using Random
using JLD
default(fmt = :png)

In [ ]:
function simpler_jump(x, y, a = 1)
    theta = rand(Uniform(0, 2 * π))
    u = rand(Uniform(0, 1))

    r = a/(3 * u)^(1/3)

    xfl = r * sin(theta)
    yfl = r * cos(theta)

    (floor(Int, x + xfl + 1/2),
    floor(Int, y + yfl + 1/2))
end

function gridoffsetandsize(infectors, removed)
    xoffset = yoffset = 0

    data = vcat(infectors, removed)

    xcoords = map(x -> x[1], data)
    ycoords = map(x -> x[2], data)

    (xmin, xmax) = (minimum(xcoords), maximum(xcoords))
    (ymin, ymax) = (minimum(ycoords), maximum(ycoords))

    if xmin < 1
        xoffset = abs(xmin)
    end

    if ymin < 1
        yoffset = abs(ymin)
    end

    gridsize = max(xmax + xoffset, ymax + yoffset)

    # add 1 because you need to include the point 0
    (xoffset + 1, yoffset + 1, gridsize + 1)
end

function creategrid(infectors, removed, xoffset, yoffset, gridsize)
    grid = zeros(gridsize, gridsize)

    infector_coords = map(x -> x[1:2], infectors)
    removed_coords = map(x -> x[1:2], removed)

    for (x, y) in removed_coords
        grid[x + xoffset, y + yoffset] = 0.5
    end

    for (x, y) in infector_coords
        grid[x + xoffset, y + yoffset] = 1
    end

    grid
    end

    function visualizegrid(grid, time)
        if 1 in grid
            colorgradient = cgrad([:black, :white, :red])
        else
            colorgradient = cgrad([:black, :white])
    end

    heatmap(1:size(grid, 1),
            1:size(grid, 2),
            grid,
            legend = :none,
            title="Time step $(time).",
            c = colorgradient,
            size=(650, 600))
end

function probinfected(x, y, r, p1, p2)
    if x^2 + y^2 < r^2
        return p1
    else
        return p2
    end
end

function outbreakwithjumps_varp(maxtime :: Int,
                           maxtau :: Int,
                           nextstepfcn :: Function,
                           probinfected :: Function,
                           p_arr :: Array{Float64, 1},
                           graphcapturetimes :: Array{Int, 1},
                           addinfectors = true) :: Tuple{Array{Int, 1},
                                                   Array{Array{Int}, 1},
                                                   Array{Array{Int}, 1},
                                                   Array{Any, 1},
                                                   Array{Int64, 1}}
    # Tracking the different agents
    x_start = 0
    y_start = 0

    newinfections = Array{Int64, 1}()
    infectors = [[x_start, y_start, 0, 0]]
    numinfected = [length(infectors)]
    removed = Set([[x_start, y_start]])
    r_inf = Array{Tuple{Int64, Int64}, 1}()
    graphcomponents = Array{Any, 1}()


    for time in 1:maxtime
        ninfectors = length(infectors)
        ninfected = 0
        newinfections_now = 0

        for i in 1:ninfectors
            infector = infectors[i]
            (infector[1], infector[2]) = simpler_jump(infector[1], infector[2])
            infector[3] += 1

            inremoved = infector[1:2] in removed

            isinfecting = rand() <= probinfected(infector[1], infector[2], p_arr[1], p_arr[2], p_arr[3])

            if !inremoved && isinfecting
                push!(removed, infector[1:2])
            end

            if !inremoved && isinfecting
                newinfections_now += 1
                if !addinfectors
                    ninfected += 1
                else
                    push!(infectors, [infector[1:2]..., 0, 0])
            end

                infector[4] += 1
            end
        end

        push!(newinfections, newinfections_now)

        infectedtoremove = filter(x -> x[3] >= maxtau, infectors)
        for infector in infectedtoremove
            push!(r_inf, (infector[4], time))
        end

        filter!(x -> x[3] < maxtau, infectors)

        if addinfectors
            push!(numinfected, length(infectors))
        else
            push!(numinfected, ninfected)
        end

        if time in graphcapturetimes
            push!(graphcomponents, [copy(time), deepcopy(infectors), collect(removed)])
        end
    end

    removed = collect(removed)

    graphs = Array{Any, 1}()
    for i in 1:length(graphcomponents)
        (xoffset, yoffset, gridsize) = gridoffsetandsize(graphcomponents[i][2], graphcomponents[i][3])
        push!(graphs, creategrid(graphcomponents[i][2], graphcomponents[i][3], xoffset, yoffset, gridsize))
    end

    (numinfected, infectors, removed, graphs, newinfections)

end

function getrnots(maxtime :: Int64,
                    tau :: Int64,
                    p :: Float64,
                    nextstepfcn :: Function,
                    probinfected :: Function,
                    #graphcapturetimes :: Array{Int, 1},
                    iterations = 10000,
                    addinfectors = false)
    rnots = Array{Int64, 1}()
    gplots = Array{Any, 1}()

    for i in 1:iterations
        (infections, infectors, removed, grids) = outbreakwithjumps_varp(
            maxtime, tau, nextstepfcn, probinfected, [1, p, p], [0], addinfectors
        )

        push!(rnots, sum(infections)-1)

        if i % 1000 == 0
            GC.gc()
            push!(gplots, grids)
        end
    end

    (gplots, rnots)
end

In [ ]:
function tf2(τ)
    if τ < 20
        maxtime = 200
    else
        maxtime = 10 * τ
    end
    return maxtime
end

rnots = Array{Any, 1}();

τs = [1, 2, 4, 8, 16, 32, 64, 128]

ps = [1, 1/2, 1/4, 1/8, 1/16, 1/32, 1/64]

arrsize = length(τs) * length(ps)
listofrnots = fill(Int[], arrsize) # essentially Array{Array{Int, 1}, 1}()
meanrnots = zeros(Float64, arrsize)
stdrnots = zeros(Float64, arrsize)

t1 = time()
@Threads.threads for i in 1:(length(τs) * length(ps)) # Hyperthreading
    τidx = floor(Int, (i-1) / length(ps)) + 1; pidx = mod(i, length(ps));
    if pidx == 0
        pidx = length(ps)
    end
    τ = τs[τidx]; p = ps[pidx]
    println(string("p = ", p, ", τ = ", τ))
    maxtime = tf2(τ)
    (_, rnots) = getrnots(maxtime, τ, p, simpler_jump, probinfected, 2_500_000, false);
    idx = (τidx-1) * length(ps) + pidx
    listofrnots[idx] = rnots
    meanrnots[idx] = mean(rnots)
    stdrnots[idx] = std(rnots)
    t2 = time()
    println(string("Finished Calculating 2,500,000 Iterations of R0 for p = ", p, ", τ = ", τ, " in ", t2-t1, " seconds."))
end

t2 = time()
t_elapsed1 = t2 - t1

exportfilename = "tf2_R0calc_largertau_2pt5M" # DO NOT include the file format at the end
# -------------------------------------------------------------------

fn = string(exportfilename, ".jld")
readme = "This data was generated with p being the inner for loop. So the second data point is the first tau value and the second p value."

save(fn, "listofrnots", listofrnots, "meanrnots", meanrnots, "stdrnots", stdrnots,
        "t_elapsed1", t_elapsed1,
        "τs", τs, "ps", ps, "readme", readme)